In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline 

In [ ]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

### get output of simulation:  units added grouped by jurisdiction

In [ ]:
units_by_jurs_sql = '''
    SELECT name, jurisdiction_id,sum(units_added) as units_added,year_simulation
      FROM urbansim.urbansim.output_temp_urbansim_lite_output
  GROUP BY name,jurisdiction_id,year_simulation
  ORDER BY name,jurisdiction_id,year_simulation'''

In [ ]:
units_by_jur = pd.read_sql(units_by_jurs_sql,mssql_engine)

### capacity from urbansim.parcel without sched developments

In [ ]:
capacity_by_jurs_sans_sched_dev_sql = '''  
    SELECT jurisdiction_id,  sum(capacity) as capacity_sans_sched_dev
      FROM urbansim.parcel
     WHERE site_id IS NULL and capacity > 0
  GROUP BY jurisdiction_id
  ORDER BY jurisdiction_id'''

In [ ]:
cap_by_jur_sans_sched =  pd.read_sql(capacity_by_jurs_sans_sched_dev_sql,mssql_engine,index_col='jurisdiction_id')

### capacity from urbansim.scheduled_development_do_not_use with sched developments only

In [ ]:
capacity_by_jurs_sched_dev_sql = '''  
    SELECT jurisdiction_id,  sum(res_units) as capacity_only_sched_dev
      FROM urbansim.urbansim.scheduled_development_do_not_use s
      JOIN urbansim.parcel p on p.parcel_id = s.parcel_id
  GROUP BY jurisdiction_id
  ORDER BY jurisdiction_id'''

In [ ]:
cap_by_jur_sched_dev =  pd.read_sql(capacity_by_jurs_sched_dev_sql,mssql_engine,index_col='jurisdiction_id')

### add capacity from urbansim.parcel and urbansim.scheduled_development_do_not_use

In [ ]:
cap_by_jur = cap_by_jur_sans_sched.join(cap_by_jur_sched_dev)

In [ ]:
cap_by_jur['capacity'] = cap_by_jur['capacity_sans_sched_dev'] + cap_by_jur['capacity_only_sched_dev']

In [ ]:
cap_by_jur = cap_by_jur.drop(['capacity_sans_sched_dev', 'capacity_only_sched_dev'],axis =1)

### for plotting, add row with "0" units for years in simulation when no new units (e.g. Del Mar)

In [ ]:
units_by_jur = units_by_jur.set_index(['jurisdiction_id','name','year_simulation'])

In [ ]:
units_by_jur = units_by_jur.unstack(['jurisdiction_id','name']).fillna(0).stack(['jurisdiction_id','name'])

In [ ]:
units_by_jur.reset_index(inplace=True,drop=False)

### sum units added over five year increments 

In [ ]:
bins = range(2015,2055,5)
names = [str(x) for x in range(2020,2055,5)]
units_by_jur['increment'] = pd.cut(units_by_jur.year_simulation, bins, labels=names)

In [ ]:
units_by_jur_inc = pd.DataFrame({'hs_added': units_by_jur.
                                            groupby(["increment", "jurisdiction_id","name"]).
                                 units_added.sum()}).reset_index()

### join units (by increment) with capacity

In [ ]:
units_by_jur_inc = units_by_jur_inc.set_index('jurisdiction_id').join(cap_by_jur)

### add starting year as 2017 with units added as 0 (for plotting)

In [ ]:
start_year = units_by_jur_inc.loc[units_by_jur_inc.increment=='2020'].copy()

In [ ]:
start_year['increment'] = 2017
start_year['hs_added'] = 0

In [ ]:
units_by_jur_inc = pd.concat([start_year,units_by_jur_inc])

### pivot for plotting so jurisdictions are columns and increments are rows with new units

In [ ]:
units_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='name', values='hs_added').\
reset_index().rename_axis(None, axis=1)
units_by_jur_pivot.fillna(0,inplace=True)
units_by_jur_pivot.set_index('increment',inplace=True)

### pivot for plotting so jurisdictions are columns and increments are rows with capacity

In [ ]:
cap_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='name', values='capacity').\
reset_index().rename_axis(None, axis=1)
cap_by_jur_pivot.fillna(0,inplace=True)
cap_by_jur_pivot.set_index('increment',inplace=True)

### calculate cumulative sum of units added by jurisdiction

In [ ]:
units_by_jur_pivot = units_by_jur_pivot.cumsum()

### plot results

#### for each jurisdiction join dataframe of new units and capacity

#### calculate remaining capacity by subtracting new units from starting capacity

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=10, ncols=2)
for j, jur in enumerate(units_by_jur.name.unique().tolist()):
    df_units_added = units_by_jur_pivot[[jur]]
    df_capacity = cap_by_jur_pivot[[jur]]
    df_plot = df_units_added.join(df_capacity, lsuffix='_new', rsuffix='_capacity')
    hs_column = jur + '_new'
    cap_column = jur + '_capacity'
    cap_plot_column = jur + '_cap'
    df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
    del  df_plot[cap_column]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,32))
   # df_plot.plot(kind='bar',ax=axes.flat[j],figsize=(10,32)))

In [ ]:
df_plot.plot(kind='bar')